Skeleton Code

In [1]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

<ipython-input-1-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-1-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [4]:
y.head()

,0
0,5
1,5
2,5
3,5
4,5


In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y.values.ravel())

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [7]:
X_train_full, X_test_full, y_train, y_test = train_test_split(df_scaled, encoded_y, test_size=0.2, random_state=42)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
import time
# Import StandardScaler
# Instantiate scaler outside the pipeline
scaler = StandardScaler()
start_time = time.time()
pipeline = Pipeline([('scaler', scaler), ('gnb', GaussianNB())]) # Use the instantiated scaler
pipeline.fit(X_train_full, y_train)
y_pred = pipeline.predict(X_test_full)
end_time = time.time()
time_taken = end_time - start_time
accuracy_score(y_test, y_pred)

0.7314751869476547

In [13]:
print(time_taken)
print(start_time)
print(end_time)

0.1468822956085205
1736584668.5079775
1736584668.6548598


In [18]:
from sklearn.cluster import KMeans
n_clusters = 50
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(df_scaled.T)  # Transpose to treat features as data points
selected_features_indices = [np.random.choice(np.where(kmeans.labels_ == i)[0]) for i in range(n_clusters)]
selected_features = df.iloc[:, selected_features_indices]

In [19]:
selected_features.head()

,210,166,172,331,159,453,514,62,60,207,...,341,118,161,380,156,371,555,238,77,188
0,-0.022897,-0.992165,0.994429,-0.999476,0.301099,0.547151,-0.796840,-1.0,-0.988849,-0.957637,...,-0.995671,0.033644,-0.055517,-0.805536,-0.331831,-1.00,0.030400,-0.038212,0.570222,-0.039007
1,-0.109366,-0.989876,0.990486,-0.999769,0.169395,-0.068178,-0.449522,-1.0,-0.990029,-0.981248,...,-0.998878,0.084104,-0.044819,-0.915987,-0.149429,-0.32,-0.007435,-0.458346,-0.831284,0.041522
2,-0.074212,-0.987868,0.989290,-0.999725,-0.350646,-0.110720,-0.879685,-1.0,-0.992104,-0.985767,...,-0.999760,-0.110026,-0.042410,-0.980133,0.184349,-0.28,0.177899,-0.520541,-0.181090,0.089630
3,0.046396,-0.991241,0.989290,-0.999775,-0.725301,-0.049728,-0.756494,-1.0,-0.980663,-0.983509,...,-0.999778,-0.083278,-0.036333,-0.893375,0.202092,-0.28,-0.012892,-0.436459,-0.991309,0.316120
4,-0.277657,-0.992882,0.993685,-0.999834,-0.724558,-0.162230,-0.481805,-1.0,-0.990611,-0.989608,...,-0.999150,0.103065,-0.037633,-0.966204,0.473251,0.04,0.122542,-0.458299,-0.408330,0.200258


In [23]:
model = GaussianNB()
Start_time = time.time()
model.fit(selected_features, encoded_y)
end_time = time.time()
print("Time taken to train the model:", end_time - Start_time, "seconds")

from sklearn.metrics import accuracy_score
y_pred = model.predict(selected_features)
accuracy = accuracy_score(encoded_y, y_pred)
print("Accuracy:", accuracy)

Time taken to train the model: 0.03129315376281738 seconds
Accuracy: 0.8029107725788901
